In [158]:
import multiprocessing
import csv

import nltk
from nltk.tokenize import RegexpTokenizer
#import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.cluster import DBSCAN

import re
import utils
from utils import tokenizer, stopwords, stemmer

In [145]:
from scipy.spatial.distance import cosine
from importlib import reload

In [154]:
reload(utils)

<module 'utils' from '/data/Match-Job-Occupations-Textual-Analysis-IK-Research/utils.py'>

In [190]:
tokenizer = RegexpTokenizer("[a-zA-Z]+[\w']+[+]*")

In [192]:
# largest distance between points for clustering:
eps = 0.1
include_title = False
include_description = True

# TODO:
title_weight = 0.5
description_weight = 1 - title_weight


def tokenize_and_stem(text):
    """
    Tokenize and stem English text
    """
    global stemmer, tokenizer
    return [stemmer.stem(token) for token in tokenizer.tokenize(text)]

In [109]:
def ilines(file1_name, file2_name, titles=False):
    """
    File line iterator
    Yields "raw" documents
    """
    global jo_ids, last_target_row, first_source_row

    # Line no -> code
    jo_ids = []
    
    # TARGET:
    with open(file2_name, mode='r') as fileobj:
        rdr = csv.reader(fileobj)

        for i, row in enumerate(rdr):
            if i == 0: ## skip header
                continue
            # industry, occ_code, occ_name, occ_descriptions
            # sample_job_titles, task1, task2, task3, task4, task5
            jo_ids.append(row[1])
            # include industry, occ_name, occ_descriptions
            # task1, task2, task3, task4, task5
            #res = row[0] + ' ' + row[2] + ' ' + row[3] + ' ' + ' '.join(row[5:])
            res = row[3]
            yield res
        last_target_row = i-1 # row #0 is the header row
        first_source_row = i # row #0 is the header row

    # SOURCE:
    with open(file1_name, mode='r') as fileobj:
        rdr = csv.reader(fileobj)

        for i, row in enumerate(rdr):
            jo_ids.append(i)
            res = row[1]
            yield res

In [201]:
#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(
    #input="filename",
    #max_df=1.1,
    #max_features=200000,
    #min_df=0.01,
    lowercase=True,
    analyzer='word',
    stop_words=stopwords,
    use_idf=True,
    tokenizer=tokenize_and_stem,
    ngram_range=(1,1))

In [202]:
%%time
tfidf_matrix = tfidf_vectorizer.fit_transform(ilines("data/File 1.csv", "data/File 2.csv"))
print("*** Vector space shape:", tfidf_matrix.shape)

*** Vector space shape: (4351, 9065)
CPU times: user 2.5 s, sys: 0 ns, total: 2.5 s
Wall time: 2.5 s


/data/Match-Job-Occupations-Textual-Analysis-IK-Research/venv/lib/python3.4/site-packages/numpy/core/fromnumeric.py:2645: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [203]:
from sklearn.metrics.pairwise import cosine_similarity

In [205]:
similarity = cosine_similarity(tfidf_matrix[first_source_row:,:], tfidf_matrix[:first_source_row,:])
print("*** Smilarity matrix shape:", similarity.shape)

*** Smilarity matrix shape: (2000, 2351)


/data/Match-Job-Occupations-Textual-Analysis-IK-Research/venv/lib/python3.4/site-packages/numpy/core/fromnumeric.py:2645: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [213]:
def take_top10(row):
    return sorted([r for r in zip(range(row.size), row.tolist()) if r[1] > 0.3],
           key=lambda r: -r[1])[:10]

In [214]:
top10 = []
for row in similarity:
    top10.append(take_top10(row))

In [209]:
for i, l in enumerate(top10, 1):
    if l != []:
        print("Line {}: ".format(i),  end="")
        print([k+2 for (k, _) in l])
    #else:
    #    print("NO MATCH")

Line 3: [653, 1784, 497, 1640, 2058, 114, 2083, 48, 2057, 575]
Line 4: [1138, 1626, 1145, 1672, 436, 1166, 1691, 708, 1433, 1643]
Line 5: [1214, 2267, 908, 991, 913, 996, 458, 607, 1725, 100]
Line 13: [11, 836, 1415, 1599, 2262, 5, 25, 1916, 2297, 1086]
Line 14: [58, 391, 46, 1509, 1893, 27, 1071, 1522, 2146, 229]
Line 15: [1575, 263, 1387, 1623, 279, 1342, 1663, 759, 1104, 1408]
Line 16: [731, 1589, 1577, 90, 1602, 2076, 652, 1783, 1575, 1300]
Line 17: [94, 666, 2277, 90, 1602, 2076, 1443, 2212, 1208, 652]
Line 18: [230, 2012, 205, 217, 2183, 1315, 1658, 2282, 218, 1895]
Line 19: [822, 2197, 2223, 2168, 2218, 649, 1780, 247, 1372, 1482]
Line 21: [693, 1379, 1744, 1151, 1400, 1608, 1315, 1658, 2282, 1313]
Line 22: [755, 1368, 1710, 693, 1379, 1744, 1058, 1343, 1894, 748]
Line 23: [755, 1368, 1710, 1311, 1697, 1200, 1190, 1310, 1696, 1234]
Line 24: [755, 1368, 1710, 368, 1051, 2023, 348, 777, 1200, 1311]
Line 25: [1529, 1934, 2141, 2317, 755, 1368, 1710, 848, 1444, 2228]
Line 26: [305]


In [215]:
for i, l in enumerate(top10, 1):
    if l != []:
        print("{}: ".format(i),  end="")
        print(", ".join(( "{} ({:.2f})".format(k+2, score) for (k, score) in l)))


14: 58 (0.30), 391 (0.30)
23: 755 (0.38), 1368 (0.38), 1710 (0.38), 1311 (0.34), 1697 (0.34)
28: 1151 (0.32), 1400 (0.32), 1608 (0.32), 1315 (0.31), 1658 (0.31), 2282 (0.31)
43: 750 (0.31), 1363 (0.31), 1705 (0.31)
50: 101 (0.34), 429 (0.34), 1091 (0.34)
62: 266 (0.37), 1516 (0.37), 2279 (0.37), 2341 (0.37)
77: 1191 (0.35), 592 (0.31), 2281 (0.31)
79: 1259 (0.33)
80: 1259 (0.32)
85: 83 (0.31), 505 (0.31), 1921 (0.31)
102: 644 (0.40), 1775 (0.40), 538 (0.38), 1150 (0.38), 1612 (0.38), 1165 (0.32), 1609 (0.32)
108: 644 (0.38), 1775 (0.38), 538 (0.35), 1150 (0.35), 1612 (0.35)
120: 706 (0.34), 1431 (0.34), 1641 (0.34), 1320 (0.31), 1656 (0.31)
123: 1151 (0.34), 1400 (0.34), 1608 (0.34), 498 (0.32), 1185 (0.32), 1828 (0.32), 1282 (0.31), 1726 (0.31), 1227 (0.31), 1223 (0.30)
124: 302 (0.31)
144: 1138 (0.42), 1626 (0.42), 1145 (0.41), 1672 (0.41), 436 (0.40), 1166 (0.40), 1691 (0.40)
157: 1136 (0.30), 1798 (0.30), 1967 (0.30)
192: 496 (0.40), 775 (0.40)
224: 1288 (0.45), 1758 (0.45), 693 (0

In [94]:
?tfidf_vectorizer.fit

In [96]:
tfidf_vectorizer.transform("""
"Design, develop, or evaluate energy-related projects or programs to reduce energy costs or improve energy efficiency during the designing, building, or remodeling stages of construction. May specialize in electrical systems; heating, ventilation, and air-conditioning (HVAC) systems; green buildings; lighting; air quality; or energy procurement.","Sample of reported job titles: Distributed Generation Project Manager, Energy Efficiency Engineer, Energy Engineer, Energy Manager, Environmental Solutions Engineer, Industrial Energy Engineer, Measurement And Verification Engineer, Test and Balance Engineer",Identify energy savings opportunities and make recommendations to achieve more energy efficient operation.,"Manage the development, design, or construction of energy conservation projects to ensure acceptability of budgets and time lines, conformance to federal and state laws, or adherence to approved specifications.","Conduct energy audits to evaluate energy use, costs, or conservation measures.",Monitor and analyze energy consumption.,"Perform energy modeling, measurement, verification, commissioning, or retro-commissioning."
""")

/data/Match-Job-Occupations-Textual-Analysis-IK-Research/venv/lib/python3.4/site-packages/numpy/core/fromnumeric.py:2645: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


<1145x10500 sparse matrix of type '<class 'numpy.float64'>'
	with 733 stored elements in Compressed Sparse Row format>